<a href="https://colab.research.google.com/github/krdlnsnks/Miuul/blob/main/Amazon_Rating_Product_Sorting_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Amazon ürün verilerini içeren bu veri seti ürün kategorileri ile çeşitli metadataları içermektedir.
# Elektronik kategorisindeki en fazla yorum alan ürünün kullanıcı puanları ve yorumları vardır.

In [2]:
# Değişkenler:
# reviewerID: Kullanıcı ID’si
# asin: Ürün ID’si
# reviewerName: Kullanıcı Adı
# helpful: Faydalı değerlendirme derecesi
# reviewText: Değerlendirme
# overall: Ürün rating’i
# summary: Değerlendirme özeti
# unixReviewTime: Değerlendirme zamanı
# reviewTime: Değerlendirme zamanı Raw
# day_diff: Değerlendirmeden itibaren geçen gün sayısı
# helpful_yes: Değerlendirmenin faydalı bulunma sayısı
# total_vote: Değerlendirmeye verilen oy sayısı

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [5]:
from google.colab import files
uploaded = files.upload()

Saving amazon_review.csv to amazon_review.csv


In [6]:
import io
df = pd.read_csv(io.BytesIO(uploaded['amazon_review.csv']))

In [7]:
df["overall"].mean() # Ürünün ortalama fiyatı

4.587589013224822

In [8]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [9]:
# Tarihe Göre Ağırlıklı Puan Ortalamasını Hesaplama

In [14]:
df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), "overall"].mean()
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25)) & (df["day_diff"] <= df["day_diff"].quantile(0.50)), "overall"].mean() # 4.64
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.50)) & (df["day_diff"] <= df["day_diff"].quantile(0.75)), "overall"].mean() # 4.57
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean()

4.4462540716612375

In [12]:
# zaman bazlı ortalama ağırlıkların belirlenmesi
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.25), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.25)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.50)), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.50)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w4 / 100

In [15]:
time_based_weighted_average(df, w1=28, w2=26, w3=24, w4=22)

4.595593165128118

In [16]:
df["overall"].mean()

4.587589013224822

In [17]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [18]:
df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]]

In [19]:
df.head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime
0,NaN,4.00000,Four Stars,0,0,0,2014-07-23
1,0mie,5.00000,MOAR SPACE!!!,0,0,0,2013-10-25
2,1K3,4.00000,nothing to really say....,0,0,0,2012-12-23
3,1m2,5.00000,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21
4,2&amp;1/2Men,5.00000,best deal around,0,0,0,2013-07-13


In [20]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [24]:
def score_up_down_diff(up, down):
    return up - down

In [22]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [25]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_pos_neg_diff", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85
...,...,...,...,...,...,...,...,...
4072,"sb21 ""sb21""",5.00000,Used for my Samsung Galaxy Tab 2 7.0,6,0,6,2012-11-09,6
2268,Jerry Saperstein,5.00000,Incredible simply incredible.,8,2,10,2014-03-17,6
121,A. Lee,5.00000,ready for use on the Galaxy S3,5,0,5,2012-05-09,5
323,"Amazon Customer ""Milo""",5.00000,Working fine in my Note 3,6,1,7,2014-01-04,5


In [26]:
# score_average_rating
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_average_rating", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.00000,Perfect!!,1,0,1,2012-12-19,1,1.00000
2881,Lou Thomas,5.00000,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00000
1073,C. Sanchez,5.00000,Tons of space for phone,1,0,1,2013-08-13,1,1.00000
445,"Apache ""Elizabeth""",4.00000,Amazon Great Prices,1,0,1,2013-12-18,1,1.00000
3923,Rock Your Roots,5.00000,What more to say?,1,0,1,2013-12-30,1,1.00000
...,...,...,...,...,...,...,...,...,...
1609,Eskimo,5.00000,Bet you wish you had one of these,7,0,7,2014-03-26,7,1.00000
402,"Andrew R. Seiler ""bookfish""",5.00000,"great, works well with galaxy s3",1,0,1,2012-09-25,1,1.00000
2226,JEFE,5.00000,great memory,1,0,1,2014-06-16,1,1.00000
3901,"Roberto K. Cho ""chorobe""",4.00000,Good while it lasted! BUT good customer support!,1,0,1,2013-03-20,1,1.00000


In [27]:
# wilson_lower_bound
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811
...,...,...,...,...,...,...,...,...,...,...
1072,Crysis Complex,5.00000,Works wonders for the Galaxy Note 2!,5,0,5,2012-05-10,5,1.00000,0.56552
2583,J. Wong,5.00000,Works Great with a GoPro 3 Black!,5,0,5,2013-08-06,5,1.00000,0.56552
121,A. Lee,5.00000,ready for use on the Galaxy S3,5,0,5,2012-05-09,5,1.00000,0.56552
1142,Daniel Pham(Danpham_X @ yahoo. com),5.00000,Great large capacity card,5,0,5,2014-02-04,5,1.00000,0.56552
